# Turbine-based ambient flow data

In some cases the inflow data is directly known at the turbine locations, in terms of (state, turbine)-type data. Such data can be simulated with _foxes_ by using the `TurbinePointCloud` states class, as demonstrated here.

We start by importing the necessary packages and setting up the engine:

In [ ]:
%matplotlib inline
import numpy as np
from xarray import Dataset, date_range
import matplotlib.pyplot as plt

import foxes
import foxes.variables as FV

In [ ]:
engine = foxes.Engine.new("default", verbosity=0)

The first step is the creation of a regular grid of wind turbines:

In [ ]:
n_turbines_x = 2
n_turbines_y = 3
n_turbines = n_turbines_x * n_turbines_y

farm = foxes.WindFarm()
foxes.input.farm_layout.add_grid(
    farm,
    xy_base=[1000, 1000],
    step_vectors=[[500, 0], [0, 500]],
    steps=[n_turbines_x, n_turbines_y],
    turbine_models=["DTU10MW"],
    verbosity=0,
)

In [ ]:
o = foxes.output.FarmLayoutOutput(farm)
o.get_figure()
plt.show()

Next, let's create an artificial Dataset object that represents data with (time, turbine) dimensions, representing a timeseries in January 2026 measured at he turbines of the above wind farm:

In [ ]:
n_times = 10
times = date_range("2026-01-01", periods=n_times, freq="10min", unit="s")
times

In [ ]:
np.random.seed(42)

# Turbine i has wind speed (10 + i.x) m/s, where x grows linear with time
ws = np.zeros((n_times, n_turbines))
ws[:] = 10 + np.arange(n_turbines)[None, :]
ws += np.linspace(0, 1, n_times, endpoint=False)[:, None]

# the wind directions are randomly selected from a southern sector:
wd = np.random.uniform(0, 190, (n_times, n_turbines))

sdata = Dataset(
    coords={"time": times},
    data_vars={
        "ws": (("time", "turbine"), ws),
        "wd": (("time", "turbine"), wd),
        "ti": ("time", 0.1 + np.arange(n_times) / 100),
        "rho": ("turbine", 1.2 + np.arange(n_turbines) / 100),
    },
)
sdata

This is what the wind speed per turbine looks like, increasing linearly with time as described above:

In [ ]:
for t in range(0, n_times):
    plt.scatter(sdata["turbine"], sdata["ws"][t])
plt.xticks(sdata["turbine"])
plt.xlabel("turbine")
plt.ylabel("ws [m/s]")
plt.show()

The data frame `sdata` is the base for our ambient states object:

In [ ]:
states = foxes.input.states.TurbinePointCloud(
    data_source=sdata,
    states_coord="time",
    turbine_coord="turbine",
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2ncvar={
        FV.WS: "ws",
        FV.WD: "wd",
        FV.TI: "ti",
        FV.RHO: "rho",
    },
)

Next, let's create the algorithm object:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm=farm,
    states=states,
    wake_models=["TurbOPark"],
    rotor_model="centre",
)

We now run the farm calculation:

In [ ]:
with engine:
    farm_results = algo.calc_farm()

The results demonstrate that all data was correctly passed from the flow states to ambient variables:

In [ ]:
farm_results.to_dataframe()[
    [FV.AMB_WD, FV.AMB_RHO, FV.AMB_TI, FV.AMB_REWS, FV.REWS, FV.P]
]

For completeness, the states results at off-turbine evaluation points will be interpolated by point cloud methods.